In [19]:
import cv2
import glob
import pandas as pd
import numpy as np
import os
from PIL import Image
import csv
import re
from sklearn.model_selection import train_test_split
from keras.models import Model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, plot_confusion_matrix

# Output Dataset

In [2]:
output_data = pd.read_csv("output columns.csv")
output_data

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11
0,4.00,4.02,2.54,0.59,0.59,119.37,116.80,65.54,10.0,100,10
1,3.96,3.96,2.54,0.60,0.60,121.43,121.57,63.32,10.0,100,12
2,4.07,4.07,2.51,0.59,0.59,114.64,116.35,65.82,10.0,10,10
3,4.01,4.06,2.48,0.59,0.59,118.76,115.70,65.57,10.0,10,12
4,4.07,4.03,2.58,0.60,0.60,119.31,122.56,63.81,10.0,10,8
...,...,...,...,...,...,...,...,...,...,...,...
1480,12.00,12.30,8.28,0.48,0.48,137.79,137.45,70.97,60.0,99,10
1481,12.48,12.61,8.74,0.48,0.48,148.18,147.91,76.82,60.0,99,8
1482,11.70,12.16,8.47,0.48,0.48,144.53,142.14,77.85,60.0,9,10
1483,12.15,11.91,8.04,0.48,0.48,142.57,145.60,71.21,60.0,9,12


# Input Dataset

In [4]:
imgs = []
for root, dirs, files in os.walk("input"):
    for name in files:
        path = os.path.join(root, name)
        img= cv2.imread(path)
        imgs.append(img)

In [5]:
img.size, img.shape, img.ndim

(410700, (370, 370, 3), 3)

In [6]:
np_data = np.array(imgs)
print(np_data.shape)

(1485, 370, 370, 3)


In [7]:
np_data = np_data.reshape(1485, -1)

In [8]:
df=pd.DataFrame(np_data, columns=[f"Col{i}" for i in range(410700)])
df

,Col0,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,...,Col410690,Col410691,Col410692,Col410693,Col410694,Col410695,Col410696,Col410697,Col410698,Col410699
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1481,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
scaler = MinMaxScaler()
input_data = scaler.fit_transform(df)

## Data Splitting

In [11]:
X_train, X_rem, y_train, y_rem = train_test_split(df,output_data['C2'], train_size=0.8)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [12]:
X_train.shape, y_train.shape, 

((1188, 410700), (1188,))

In [13]:
X_test.shape, y_test.shape

((149, 410700), (149,))

In [14]:
X_valid.shape, y_valid.shape

((148, 410700), (148,))

# CNN Model

In [15]:
model = keras.models.load_model('model.h5')
model.layers

In [17]:
for layer in model.layers:
    layer.trainable = False

In [20]:
t = Model(model.input, model.output, name='Model')

In [21]:
t.compile(optimizer=Adam(),loss = tf.keras.losses.MeanAbsolutePercentageError(), metrics= ['mse'])
t.summary()

Model: "Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_input (InputLayer)   [(None, 410700, 1)]       0         
                                                                 
 conv1d (Conv1D)             (None, 410699, 4)         12        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 205349, 4)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 821396)            0         
                                                                 
 dense (Dense)               (None, 8)                 6571176   
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                             

In [22]:
cnn_history = t.fit(X_train, y_train, batch_size=10, epochs = 5, validation_data=(X_valid,y_valid),verbose=1)

Epoch 1/5
119/119 [==============================] - 45s 374ms/step - loss: 84.5167 - mse: 48.5218 - val_loss: 84.9307 - val_mse: 52.6223
Epoch 2/5
119/119 [==============================] - 44s 367ms/step - loss: 84.5167 - mse: 48.5218 - val_loss: 84.9307 - val_mse: 52.6223
Epoch 3/5
119/119 [==============================] - 43s 361ms/step - loss: 84.5167 - mse: 48.5218 - val_loss: 84.9307 - val_mse: 52.6223
Epoch 4/5
119/119 [==============================] - 43s 362ms/step - loss: 84.5167 - mse: 48.5218 - val_loss: 84.9307 - val_mse: 52.6223
Epoch 5/5
119/119 [==============================] - 44s 371ms/step - loss: 84.5167 - mse: 48.5218 - val_loss: 84.9307 - val_mse: 52.6223
